#  <center> 双循环ESN复现代码

In [1]:
import numpy as np
import sys
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn import preprocessing
import warnings
warnings.filterwarnings('ignore')
from ESN_for_test import ESN
%matplotlib inline

## <center> 【读取数据集部分】

In [2]:
min_max_scaler = preprocessing.MinMaxScaler()
data_name = "output"
data_path = r"datasets/Data.csv"
with open(data_path,encoding='utf-8') as f:
    data = pd.read_csv(data_path,sep=",",skiprows=[0])
    data1 = data.iloc[:, [3]]
    data = data1
    data = np.array(data).astype('float64')
    data.reshape(1,-1)

    print(data)
    print("本次训练的{}数据集大小为{}".format(data_name,len(data)))

[[200.  ]
 [200.  ]
 [200.  ]
 ...
 [401.76]
 [401.98]
 [402.2 ]]
本次训练的output数据集大小为205200


In [3]:
def MSE(yhat, y):
    return np.sqrt(np.mean((yhat.flatten() - y)**2))

## <center> 训练

In [ ]:
n_reservoir= 500
sparsity   = 0.15
rand_seed  = 23

radius_set = [0.3]
noise_set = [0.01]

# radius_set = [0.5, 0.7, 0.9,  1,  1.1, 1.3, 1.5]
# noise_set = [ 0.01, 0.03,0.07, 0.1, 0.3, 0.5, 0.7,0.9]

radius_set_size  = len(radius_set)
noise_set_size = len(noise_set)

trainlen = 40000
future = 50
futureTotal = 3000

loss = np.zeros([radius_set_size, noise_set_size])
MSE_output = []

for l in range(radius_set_size):
    rho = radius_set[l]
    for j in range(noise_set_size):
        noise = noise_set[j]
        pred_tot=np.zeros(futureTotal)
        esn = ESN(n_inputs = 1,
          n_outputs = 1,
          n_reservoir = n_reservoir,
          sparsity=sparsity,
          random_state=rand_seed,
          spectral_radius = rho,
          noise=noise)
        for index,i in enumerate(range(0,futureTotal,future)):
            pred_training = esn.fit(np.ones(trainlen),data[i:trainlen+i])
            prediction = esn.predict(np.ones(future))
            pred_tot[i:i+future] = prediction[:,0]
            print("\r", end="")
            print("Training progress: {}/{}".format(index,(futureTotal/future)), end="")
            sys.stdout.flush()

        loss[l, j] = MSE(pred_tot, data[trainlen:trainlen+futureTotal])
        print('rho = ', radius_set[l], ', noise = ', noise_set[j], ', MSE = ', loss[l][j] )


Training progress: 27/60.0

## <center> 预测

In [ ]:
import seaborn as sns
from matplotlib import rc
rc('text', usetex=False)


for rand_seed in range(1):
    n_reservoir= 500
    sparsity = 0.2
    rand_seed=rand_seed
    spectral_radius = 0.8
    noise = 0.01

    esn = ESN(n_inputs = 1,
          n_outputs = 1,
          n_reservoir = n_reservoir,
          sparsity=sparsity,
          random_state=rand_seed,
          spectral_radius = spectral_radius,
          noise=noise)

    trainlen = 160000
    future = 200
    futureTotal = 2000
    pred_tot=np.zeros(futureTotal)

    for index,i in enumerate(range(0,futureTotal,future)):
        pred_training = esn.fit(np.ones(trainlen),data[i:trainlen+i])
        prediction = esn.predict(np.ones(future))
        pred_tot[i:i+future] = prediction[:,0]
        print("Training progress: {}/{}".format(index,(futureTotal/future)), end="")
        sys.stdout.flush()
    mse = MSE(pred_tot, data[trainlen:trainlen+futureTotal])
    print(mse)
    esn.print_sth()

    show_total = 4000

    plt.figure(figsize=(30,8))
    plt.plot(range(trainlen,trainlen+show_total),data[trainlen:trainlen+show_total],'b',label="Data", alpha=0.3)
    #plt.plot(range(0,trainlen),pred_training,'.g',  alpha=0.3)
    plt.plot(range(trainlen,trainlen+show_total),pred_tot[:show_total],'k',  alpha=0.8, label='Free Running ESN')
    lo,hi = plt.ylim()
    plt.plot([trainlen,trainlen],[lo+np.spacing(1),hi-np.spacing(1)],'k:', linewidth=4)
    plt.savefig("save_data//" + str(rand_seed) +'.png')
    sns.despine()

In [ ]:
show_total = 2000
plt.figure(figsize=(30,8))
plt.plot(range(trainlen,trainlen+show_total),data[trainlen:trainlen+show_total],'b',label="Data", alpha=0.3)
#plt.plot(range(0,trainlen),pred_training,'.g',  alpha=0.3)
plt.plot(range(trainlen,trainlen+show_total),pred_tot[:show_total],'k',  alpha=0.8, label='Free Running ESN')
lo,hi = plt.ylim()
plt.plot([trainlen,trainlen],[lo+np.spacing(1),hi-np.spacing(1)],'k:', linewidth=4)
plt.savefig("save_data//" + str(rand_seed) +"gg"+'.png')
sns.despine()

In [ ]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(pred_tot[:show_total],data[trainlen:trainlen+futureTotal])
print(mse)